In [5]:
conda install -c conda-forge glpk

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda





Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\emok\Anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    glpk-5.0                   |       h8ffe710_0         3.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  glpk               conda-forge/win-64::glpk-5.0-h8ffe710_0 None




glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    | 7          |   7% 
glpk-5.0             | 3.3 MB    | #2         |  12% 
glpk-5.0             | 3.3 MB    | #9         |  20% 
glpk-5.0             | 3.3 MB    | ##5        |  25% 
glpk-5.0             | 3.3 MB    | ###2       |  33% 
glp

In [4]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.core import Constraint
from pyomo.dae import *

Static model, when alpha = 0

In [68]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdata.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())

# Define the constraint
model.d = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of >= data['d'][h])

# Define the objective function
model.Z = Objective(expr=model.k_ipv * model.N_pv +
                   model.k_ion * model.N_on +
                   model.k_iof * model.N_of)

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())


N_pv = 32315.4468498913
N_on = 352866.485014811
N_of = 517826.219014595
Z = 35698618.499111354


Dynamic model, when alpha>0

In [27]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.k_vv2g = Param(default=226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)
model.E_s = Param(default=160425)
model.alpha = Param(default=0.5)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals)
model.V_charge = Var(model.hours, domain=NonNegativeReals)
model.V_discharge = Var(model.hours, domain=NonNegativeReals)
model.C = Var(model.hours, domain=NonNegativeReals)

# Market balance constraints
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h])

model.d2 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of == data['d'][h] + model.V_charge[h] + model.C[h])


#State of charge constraint
def s1_constraint(model,h):
    if h == model.hours.first():
        return model.V[h] == model.alpha*model.E_s / 2
    else: 
        return model.V[h] == model.V[h-1] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h]
model.s1 = Constraint(model.hours, rule = s1_constraint)

"""No free lunch"""
model.s2 = Constraint(rule=lambda model:  model.V[model.hours.first()] == model.V[model.hours.last()])

"""Maximum energy storage must be less or equal to capacity"""
model.s3 = Constraint(model.hours, rule=lambda model, h: model.V[h] <= model.alpha*model.E_s)

# Charging and discharging constraints
""" Maximum discharge rate within a single hour """
model.disc1 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.alpha*data['pv2g'][h])

""" Maximum charge rate within a single hour """
model.char1 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.alpha*data['pv2g'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model), sense=minimize) # Use the defined objective expression with the 'h' variable

# Solve the optimization problem
solver = SolverFactory('glpk')
#solver.solve(model)
results = solver.solve(model,tee=True)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())



GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\emok\AppData\Local\Temp\tmplee4r02c.glpk.raw --wglp C:\Users\emok\AppData\Local\Temp\tmpk5ano_1p.glpk.glp
 --cpxlp C:\Users\emok\AppData\Local\Temp\tmpye6wx9px.pyomo.lp
Reading problem data from 'C:\Users\emok\AppData\Local\Temp\tmpye6wx9px.pyomo.lp'...
52562 rows, 26284 columns, 131400 non-zeros
324140 lines were read
Writing problem data to 'C:\Users\emok\AppData\Local\Temp\tmpk5ano_1p.glpk.glp'...
271573 lines were written
GLPK Simplex Optimizer 5.0
52562 rows, 26284 columns, 131400 non-zeros
Preprocessing...
26279 rows, 26279 columns, 105112 non-zeros
Scaling...
 A: min|aij| =  2.860e-05  max|aij| =  1.107e+00  ratio =  3.872e+04
GM: min|aij| =  7.727e-03  max|aij| =  1.294e+02  ratio =  1.675e+04
EQ: min|aij| =  5.971e-05  max|aij| =  1.000e+00  ratio =  1.675e+04
Constructing initial basis...
Size of triangular part is 26278
      0: obj =  -7.876027950e+09 inf =   1.806e+11 (24250)
   32

ValueError: No value for uninitialized NumericValue object V_discharge[0]

In [13]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.k_vv2g = Param(default=226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)
model.alpha = Param(default=0.5)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals, bounds=(0, model.alpha*160425), initialize=0.5*model.alpha*160425)
model.V_prev = Param(model.hours, initialize=lambda model, h: model.V[h-1] if h > 0 else 0, within=NonNegativeReals)
model.V_charge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, model.alpha*data['pv2g'][h]))
model.V_discharge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, model.alpha*data['pv2g'][h]))
model.C = Var(model.hours, domain=NonNegativeReals)



# Define the constraint
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h])

model.d2 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of == data['d'][h] + model.V_charge[h] + model.C[h])


# Update the constraint using model.V_prev
#model.d3 = Constraint(model.hours, rule=lambda model, h: model.V[h] == model.V_prev[h] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h] if h > 0 else model.V[h] == 0)


model.batt_balance = ConstraintList()
for h in model.hours:
    if h == model.hours.first:
        model.batt_balance.add(model.V[h] == 0.5*model.alpha*160425)  # stored energy of the battery set as half of rated capacity at 1st hour
    elif h == model.hours.last:
        model.batt_balance.add(model.V[h] == 0.5*model.alpha*160425) 
    else:
        model.batt_balance.add(model.V[h] == model.V_prev[h] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model)) # Use the defined objective expression with the 'h' variable

# Solve the optimization problem
solver = SolverFactory('glpk')
#solver.solve(model)
results = solver.solve(model,tee=True)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\emok\AppData\Local\Temp\tmpbbkyd2wj.glpk.raw --wglp C:\Users\emok\AppData\Local\Temp\tmptn175xv3.glpk.glp
 --cpxlp C:\Users\emok\AppData\Local\Temp\tmpalyg29g5.pyomo.lp
Reading problem data from 'C:\Users\emok\AppData\Local\Temp\tmpalyg29g5.pyomo.lp'...
26281 rows, 35044 columns, 105121 non-zeros
227778 lines were read
Writing problem data to 'C:\Users\emok\AppData\Local\Temp\tmptn175xv3.glpk.glp'...
227773 lines were written
GLPK Simplex Optimizer 5.0
26281 rows, 35044 columns, 105121 non-zeros
Preprocessing...
17521 rows, 8764 columns, 61323 non-zeros
Scaling...
 A: min|aij| =  2.860e-05  max|aij| =  1.107e+00  ratio =  3.872e+04
GM: min|aij| =  7.727e-03  max|aij| =  1.294e+02  ratio =  1.675e+04
EQ: min|aij| =  5.971e-05  max|aij| =  1.000e+00  ratio =  1.675e+04
Constructing initial basis...
Size of triangular part is 17521
      0: obj =   4.280367168e+09 inf =   8.364e+05 (10689)
     57

ValueError: No value for uninitialized NumericValue object V_discharge[0]

In [ ]:
# Water balance equation
def storage_balance_rule(model, h):
    if h == model.hours:  # final volume is same as initial volume at 50%
        return model.V[h] == 0.5*model.alpha*160425
    elif h > 1:
        cascade_inflow = 0
        for stor in model.res:
            # connectMat is a matrix that has a 1 where there is a connection and 0 where there is not
            cascade_inflow = cascade_inflow + model.connectMat[stor, r]*(model.turbine[t, stor]/model.slope[stor]+model.spilledFlow[t, stor])
            if model.connectMat[stor, r] == 1:
                print(stor, r, t, value(cascade_inflow))  # this always prints out 0.0 for cascade_inflow
         discharged_flow = model.turbine[t, r]/model.slope[r]  # model.turbine is in MW: divide by slope to get discharge flow [m3/s]
         print(value(discharged_flow))  # this always prints out 0.0
         return model.volume[t, r] == model.volume[t-1, r]+(cascade_inflow+model.inflow[t, r]-model.spilledFlow[t, r]-discharged_flow)*model.secPerTimeStep
    else:
        # start volume constrained to 60% of max volume
        return model.volume[t, r] == model.max_Vol[r]*0.6
m.water_balance = Constraint(m.stage, m.res, rule=water_balance_rule)


In [ ]:
model.batt_balance = ConstraintList()
        for h in model.hours:
            if h == 0:
                model.batt_balance.add(
                    model.V[h] == 0.5*model.alpha*160425)  # stored energy of the battery set as half of rated capacity at 1st hour
            else:
                model.batt_balance.add(
                    model.V[h] == model.V[h - 1] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h]
                # model.e_cons.add(
                #   model.e_batt[i] == model.e_batt[i-1] - (model.p_char[i] + model.p_disc[i]) * model.k[i] * dt)
                # energy balance of battery:
                # stored energy = the energy level of last hour + charged or discharged energy during dt - energy loss

In [7]:
# Load data into a Pandas DataFrame
data = pd.read_csv(r'D:\Polit\Seminar (Energy Econ)\dCFdatavd.csv')

# Create a Pyomo model
model = ConcreteModel()

# Define the sets
model.hours = Set(initialize=data.index, doc='Set of hours')

# Define the parameters
model.k_ipv = Param(default=40)
model.k_ion = Param(default=30)
model.k_iof = Param(default=46)
model.k_vv2g = Param(default=226.56)
model.eta_charge = Param (default=0.924)
model.eta_discharge = Param(default=0.903)
model.alpha = Param(default=0.5)

# Define the variables
model.N_pv = Var(domain=NonNegativeReals, initialize=0)
model.N_on = Var(domain=NonNegativeReals, initialize=0)
model.N_of = Var(domain=NonNegativeReals, initialize=0)
model.V = Var(model.hours, domain=NonNegativeReals, bounds=(0, model.alpha*160425), initialize=0.5*model.alpha*160425)
model.V_prev = Param(model.hours, initialize=lambda model, h: model.V[h-1] if h > 0 else 0, within=NonNegativeReals)
model.V_charge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, model.alpha*data['pv2g'][h]))
model.V_discharge = Var(model.hours, domain=NonNegativeReals, bounds=lambda model, h:(0, model.alpha*data['pv2g'][h]))
model.C = Var(model.hours, domain=NonNegativeReals)

#Define data
model.CF_pv = Param(model.hours, initialize=data['CF_pv'].to_dict())
model.CF_on = Param(model.hours, initialize=data['CF_on'].to_dict())
model.CF_of = Param(model.hours, initialize=data['CF_of'].to_dict())
model.pv2g = Param(model.hours, initialize=data['pv2g'].to_dict())
model.d_ev = Param(model.hours, initialize=data['d_ev'].to_dict())

# Define the constraint
model.d1 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of + model.V_discharge[h] == data['d'][h])

model.d2 = Constraint(model.hours, rule=lambda model, h: model.CF_pv[h] * model.N_pv +
                                                   model.CF_on[h] * model.N_on +
                                                   model.CF_of[h] * model.N_of == data['d'][h] + model.V_discharge[h] + model.C[h])


# Update the constraint using model.V_prev
#model.d3 = Constraint(model.hours, rule=lambda model, h: model.V[h] == model.V_prev[h] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h] if h > 0 else model.V[h] == 0)


model.batt_balance = ConstraintList()
for h in model.hours:
    if h == model.hours.first:
        model.batt_balance.add(model.V[h] == 0.5*model.alpha*160425)  # stored energy of the battery set as half of rated capacity at 1st hour
    elif h == model.hours.last:
        model.batt_balance.add(model.V[h] == 0.5*model.alpha*160425) 
    else:
        model.batt_balance.add(model.V[h] == model.V_prev[h] + model.eta_charge * model.V_charge[h] - model.V_discharge[h]/model.eta_discharge - data['d_ev'][h])

#Discharge and charge constraints
""" Maximum discharge rate within a single hour """
model.disc1 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.alpha*data['pv2g'][h])

""" Sets the maximum energy available to be discharged as the SOC - minimum SOC """
model.disc2 = Constraint(model.hours, rule=lambda model, h: model.V_discharge[h] <= model.V[h] - data['d_ev'][h])

""" Maximum charge rate within a single hour """
model.char1 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.alpha*data['pv2g'][h])

""" Sets the maximum energy available to be discharged as the SOC - minimum SOC """
#model.char2 = Constraint(model.hours, rule=lambda model, h: model.V_charge[h] <= model.V[h] - data['d_ev'][h])

# Define objective function
def obj_expression(model):
    return model.k_ipv * model.N_pv + model.k_ion * model.N_on + model.k_iof * model.N_of + model.k_vv2g * sum(model.V_discharge[h] for h in model.hours)

model.Z = Objective(expr=obj_expression(model)) # Use the defined objective expression with the 'h' variable

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)

# Access the solution
print("N_pv =", model.N_pv())
print("N_on =", model.N_on())
print("N_of =", model.N_of())
print("Z =", model.Z())

N_pv = 0
N_on = 0
N_of = 0
ERROR: evaluating object as numeric value: V_discharge[0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object V_discharge[0]


ValueError: No value for uninitialized NumericValue object V_discharge[0]